<a href="https://colab.research.google.com/github/OxanaMakhneva/DS/blob/master/correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import itertools
from collections import Counter
from itertools import combinations

In [ ]:
""" Семейство функция для оценки мультиколлинеарности"""

#Функция, которая находит максимально часто задействованный в корреляции столбец
#и список столбцов, с ним коррелированных
#На входпринимает список названий столбцов, которые надо проверить на корреляцию
def calc_max_corr_col(list_of_cols, df):
    #Находим все комбинации столбцов
    col_combs = list(itertools.combinations(list_of_cols, 2))
    #Задаем списки для всех коррелированных столбцов
    col_comb_with_corr = []
    all_cols_with_corr = []
    new_list = []

    for col_comb in col_combs:
        try:
            corr_value = df[col_comb[0]].corr(df[col_comb[1]])
            #Если корреляция выше порога добавляем столбцы в списки коррелированных
            if abs(corr_value) > 0.7:
                col_comb_with_corr.append([corr_value, col_comb[0], col_comb[1]])
                all_cols_with_corr.append(col_comb[0])
                all_cols_with_corr.append(col_comb[1])

        except:
            pass

    #Считаем количество вхождений каждого столбца в списки коррелированных
    count = Counter(all_cols_with_corr)

    #Ищем столбец, наиболее часто встреччающийся в списке коррелированных
    try:
        max_corr_col = max(count, key=count.get)
    except:
        return None

    #Формируем список столбцов, с которыми он коррелирован
    for comb in col_comb_with_corr:
        if comb[1] == max_corr_col:
            new_list.append(comb[2])
        elif comb[2] == max_corr_col:
            new_list.append(comb[1])
        else:
            pass

    friends = {max_corr_col:  new_list}

    #Возвращаем название столбца, число его вхождений в список коррелированных, его друзей по корреляции
    return (max_corr_col, count[max_corr_col], friends)


 #Функция, которая определяет список мультиколленеарных столбцов
def calc_corr_cols(list_of_cols, df):
    #Список столбцов на удаление из списка комбинаций
    list_of_cols_for_drop = []
    #Список столбцов-друзей максимально коррелированныхстолбцов
    list_of_friends = []
    #Максимальное количество итераций
    number_iters = len(list_of_cols)

    for step in range(number_iters):
        #Вычисляем наиболее часто коррелированный столбец и его друзей-корелянтов
        results = calc_max_corr_col(list_of_cols, df)
        if not results:
            break
        (max_corr_col, max_corr_col_count, friens) = results

        #Если количество столбцов, с которым коррелирован самый коррелированный меньше 1 выходим из цикла
        if max_corr_col_count < 1:
            break
        else:
            #Заисываем столбец в список на удаление
            list_of_cols_for_drop.append(max_corr_col)
            #Убираем его из списка для построения комбинаций
            list_of_cols.remove(max_corr_col)
            #Фиксируем друзей-коррелянтов
            list_of_friends.append(friens)
    return (list_of_cols_for_drop, list_of_friends)

In [ ]:
#Определение коллинеарных столбцов
list_of_cols = list(df.columns)
list_of_cols_for_drop, list_of_friends = calc_corr_cols(list_of_cols, df)

#Список всех коррелированных столбцов
display(list_of_cols_for_drop)

In [ ]:
#Список всех коррелированных столбцов с указанием столбцов, с которыми они коррелированы
display(list_of_friends)